In [1]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.7/548.7 kB 2.6 MB/s eta 0:00:00a 0:00:01


In [197]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedGroupKFold
import timm
from tqdm import tqdm
import albumentations as A
from collections import defaultdict
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score,precision_score,recall_score
from torch.cuda import amp
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import gc
import copy

In [198]:
class CFG:
    seed          = 42
    debug         = False # set debug=False for Full Training
    model         =  'swin_tiny_patch4_window7_224'
    batch_size    = 32
    img_size      = [224, 224]
    epochs        = 5
    lr            = 1e-4
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-4
    T_max         = 449
    T_0           = 25
    warmup_epochs = 0
    wd            = 1e-6
    n_accumulate  = 1
    n_fold        = 5
    folds         = [0, 1, 2, 3]
    device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [199]:
df = pd.read_csv('../input/old-ultra-sound-covid-detection/covid_data.csv')

def correct_path(idx):
    path = '../input/old-ultra-sound-covid-detection/data/'+idx[6::]
    return path

df['path'] = df['path'].apply(correct_path)

In [200]:
df['fold'] = -1
skf = StratifiedGroupKFold(n_splits=4,shuffle=True,random_state=42)
for fold,(train_idx,test_idx) in enumerate(skf.split(df['path'],df['label'],groups=df['video_id'])):
    df.loc[test_idx,'fold'] = fold

In [201]:
#df = df.head(1024)

In [202]:
df['video_id'].unique()

array(['Video_0', 'Video_1', 'Video_2', 'Video_3', 'Video_4', 'Video_5',
       'Video_6', 'Video_7', 'Video_8', 'Video_9', 'Video_10', 'Video_11',
       'Video_12', 'Video_13', 'Video_14', 'Video_15', 'Video_16',
       'Video_17', 'Video_18', 'Video_19', 'Video_20', 'Video_21',
       'Video_22', 'Video_23', 'Video_24', 'Video_25', 'Video_26',
       'Video_27', 'Video_28', 'Video_29', 'Video_30', 'Video_31',
       'Video_32', 'Video_33', 'Video_34', 'Video_35', 'Video_36',
       'Video_37', 'Video_38', 'Video_39', 'Video_40', 'Video_41',
       'Video_42', 'Video_43', 'Video_44', 'Video_45', 'Video_46',
       'Video_47', 'Video_48', 'Video_49', 'Video_50', 'Video_51',
       'Video_52', 'Video_53', 'Video_54', 'Video_55', 'Video_56',
       'Video_57', 'Video_58', 'Video_59', 'Video_60', 'Video_61',
       'Video_62', 'Video_63', 'Video_64', 'Video_65', 'Video_66',
       'Video_67', 'Video_68', 'Video_69', 'Video_70', 'Video_71',
       'Video_72', 'Video_73', 'Video_74', 'Vide

In [203]:
class CovidDataset(Dataset):    
    def __init__(self,df,transforms=None,is_valid=False):
        self.df = df
        self.transforms = transforms
        self.is_valid = is_valid
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img = cv2.imread(self.df['path'].iloc[idx])
        label = self.df['label'].iloc[idx]
        
        if self.transforms:
            img = self.transforms(image=img)['image']
        #img = torch.tensor(img,dtype=torch.float32)
        img = img.to(torch.float32)
        label = torch.tensor(label)
        
        if self.is_valid:
            return img,label,self.df['video_id'].iloc[idx]
        return img,label

In [204]:
def get_transforms(data):
    
    if data == 'train':
        return A.Compose([
            A.Resize(*CFG.img_size,interpolation=cv2.INTER_NEAREST),
            #A.RandomBrightnessContrast(),
            #A.CLAHE(),
            #A.GaussNoise(),
            #A.CenterCrop(*CFG.img_size),
            A.Normalize(),
            A.HorizontalFlip(),
            ToTensorV2(),
        ],p=1.0)

    elif data == 'valid':
        return A.Compose([
            A.Resize(*CFG.img_size,interpolation=cv2.INTER_NEAREST),
            #A.CenterCrop(*CFG.img_size),
            A.Normalize(),
            ToTensorV2(),
        ],p=1.0)

In [205]:
class BaseModel(nn.Module):
    def __init__(self,cfg,pretrained=False):
        super().__init__()
        self.cfg = cfg
        self.model = timm.create_model(cfg.model,pretrained=pretrained)
        self.model.head = nn.Linear(self.model.head.in_features,3)
        
    def forward(self,x):
        output = self.model(x)
        return output

In [206]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    epoch_loss = 0
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (images, labels) in pbar:
        labels = labels.type(torch.LongTensor)
        images = images.to(device, dtype=torch.float)
        labels  = labels.to(device)
        
        batch_size = images.size(0)
        
        with amp.autocast(enabled=True):
            y_pred = model(images)
            loss   = criterion(y_pred, labels)
            loss   = loss / CFG.n_accumulate
            
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
    
        if (step + 1) % CFG.n_accumulate == 0:
            scaler.step(optimizer)
            scaler.update()
            # zero the parameter gradients
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        #print(epoch_loss)
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')
        torch.cuda.empty_cache()
        gc.collect()
    
    return epoch_loss

In [207]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch,df,fold):
    model.eval()
    epoch_loss = 0.0
    
    dataset_size = 0
    running_loss = 0.0
    
    val_scores = []
    video_ids_list = list(df[df['fold'] == fold]['video_id'].unique())
    video_ids_score = {}
    video_ids_count = {}
    for i in video_ids_list:
        video_ids_score[i] = 0
        
    for i in video_ids_list:
        df_1 = df[df['fold'] == fold].copy()
        video_ids_count[i] = len(df_1[df['video_id']==i])
        
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    oof = pd.DataFrame()
    scores = np.array([])
    real_labels = np.array([])
    for step, (images, labels, video_ids) in pbar:
        #images = images.type(torch.LongTensor)
        labels = labels.type(torch.LongTensor)
        images  = images.to(device, dtype=torch.float)
        labels   = labels.to(device)
        
        
        batch_size = images.size(0)
        
        y_pred  = model(images)
        #print(y_pred)
        #print(labels)
        loss    = criterion(y_pred, labels)
        
        
            
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        preds = y_pred.cpu().detach().numpy()
        preds = np.argmax(preds,axis=1).reshape(-1)
        #print(scores)
        scores = np.concatenate([scores,preds])
        real_labels = np.concatenate([real_labels,labels.cpu().numpy()])
        
        
        #y_pred = nn.Softmax()(y_pred)
        
        for i in range(len(video_ids)):
            #video_ids_count[video_ids[i]] +=1
            video_ids_score[video_ids[i]] += y_pred[i].cpu().detach().numpy()
            
        #val_dice = dice_coef(masks, y_pred).cpu().detach().numpy()
        #val_jaccard = iou_coef(masks, y_pred).cpu().detach().numpy()
        #val_scores.append([val_dice, val_jaccard])
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')
    '''
    scores = []
    labels = []
    for ids in video_ids_score:
        video_ids_score[ids] = video_ids_score[ids]/video_ids_count[ids]
    
    for ids in video_ids_score:
        video_ids_score[ids] = np.argmax(video_ids_score[ids])
    
    for ids in video_ids_score:
        labels.append(df[df['video_id']==ids]['label'].iloc[0])
        scores.append(float(video_ids_score[ids]))
    scores = np.array(scores)
    '''
    #val_auc = roc_auc_score(labels,list(scores))
    
    labels = real_labels
    print(labels.shape)
    print(scores.shape)
    val_scores = f1_score(labels,scores, average='macro')
    val_accuracy = accuracy_score(labels,scores)
    precision = precision_score(labels,scores, average='macro')
    recall = recall_score(labels,scores, average='macro')
    video_ids_1 = []
    for ids in video_ids_score:
        video_ids_1.append(ids)
    #oof['video_id'] = video_ids_1
    #oof['preds'] = scores
    #oof['labels'] = labels
    torch.cuda.empty_cache()
    gc.collect()
    
    return epoch_loss, val_scores, val_accuracy, precision, recall

In [208]:
def run_training(model, optimizer, scheduler, device, num_epochs,df,fold):
    
    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_score      = -np.inf
    best_epoch     = -1
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader,
                                     device=CFG.device, epoch=epoch)
        val_loss, val_scores, val_accuracy, precision, recall  = valid_one_epoch(model, valid_loader, 
                                                 device=CFG.device, 
                                                 epoch=epoch,df=df,fold=fold)
    
        history['Train Loss'].append(train_loss)
        history['Valid Loss'].append(val_loss)
        history['Valid Score'].append(val_scores)
        
        
        #print(f'thresold: {best_th:0.4f}')
        print(f'Valid F1 Score: {val_scores:0.4f}')
        
        print(f'Valid Accuracy Score: {val_accuracy:0.4f}')
        #print(f'Valid AUC Score: {val_auc:0.4f}')
        
        print(f'Valid Precision Score: {precision:0.4f}')
        print(f'Valid Recall Score: {recall:0.4f}')
        
        # deep copy the model
        if val_scores >= best_score:
            print(f"Valid Score Improved ({best_score:0.4f} ---> {val_scores:0.4f})")
            #best_oof = oof
            best_score    = val_scores
            best_epoch   = epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"best_epoch-{fold:02d}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved")
            
        last_model_wts = copy.deepcopy(model.state_dict())
        PATH = f"last_epoch-{fold:02d}.bin"
        torch.save(model.state_dict(), PATH)
            
        print(); print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    #best_oof.to_csv(f'{fold}_oof.csv',index=False)
    print("Best Score: {:.4f}".format(best_score))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [209]:
def fetch_scheduler(optimizer):
    if CFG.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CFG.T_max, 
                                                   eta_min=CFG.min_lr)
    elif CFG.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CFG.T_0, 
                                                             eta_min=CFG.min_lr)
    elif CFG.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='min',
                                                   factor=0.1,
                                                   patience=7,
                                                   threshold=0.0001,
                                                   min_lr=CFG.min_lr,)
    elif CFG.scheduler == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
    elif CFG.scheduler == 'None':
        #scheduler = None
        return None
        
    return scheduler

In [210]:
criterion = nn.CrossEntropyLoss()

In [ ]:
for fold in CFG.folds:
    print(f'#'*15)
    print(f'### Fold: {fold}')
    print(f'#'*15)
    train_df = df[df['fold'] !=fold].reset_index(drop=True)
    valid_df = df[df['fold'] ==fold].reset_index(drop=True)
    if CFG.debug:
        train_df = train_df.head()
        valid_df = valid_df.head()
    
    train_dataset = CovidDataset(train_df,transforms=get_transforms(data='train'),is_valid=False)
    valid_dataset = CovidDataset(valid_df,transforms=get_transforms(data='valid'),is_valid=True)
    
    train_loader = DataLoader(train_dataset,batch_size=CFG.batch_size,shuffle=True,num_workers=2,drop_last=True)
    valid_loader = DataLoader(valid_dataset,batch_size=CFG.batch_size,shuffle=False,num_workers=2,drop_last=False)
    
    cfg = CFG()
    
    model     = BaseModel(cfg,pretrained=True)
    model.to(CFG.device)
    optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
    scheduler = fetch_scheduler(optimizer)
    model, history = run_training(model, optimizer, scheduler,
                                  device=CFG.device,
                                  num_epochs=CFG.epochs,
                                 df=df,
                                 fold=fold)

###############
### Fold: 0
###############
cuda: Tesla P100-PCIE-16GB

Epoch 1/5

Train :   0%|          | 0/449 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
Train : 100%|██████████| 449/449 [03:50<00:00,  1.95it/s, gpu_mem=4.03 GB, lr=0.00010, train_loss=0.2049]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
Valid : 100%|██████████| 175/175 [00:30<00:00,  5.83it/s, gpu_memory=2.86 GB, lr=0.00010, valid_loss=0.6684]


(5589,)
(5589,)
Valid F1 Score: 0.8617
Valid Accuracy Score: 0.8980
Valid Precision Score: 0.9343
Valid Recall Score: 0.8256
Valid Score Improved (-inf ---> 0.8617)
Model Saved


Epoch 2/5

Train : 100%|██████████| 449/449 [03:51<00:00,  1.94it/s, gpu_mem=4.14 GB, lr=0.00010, train_loss=0.2529]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
Valid : 100%|██████████| 175/175 [00:30<00:00,  5.83it/s, gpu_memory=2.83 GB, lr=0.00010, valid_loss=0.7166]


(5589,)
(5589,)
Valid F1 Score: 0.7783
Valid Accuracy Score: 0.8250
Valid Precision Score: 0.7995
Valid Recall Score: 0.7932


Epoch 3/5

Train :  46%|████▌     | 206/449 [01:47<02:06,  1.92it/s, gpu_mem=4.15 GB, lr=0.00010, train_loss=0.1231]